In [1]:
import logging
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    MeshcatVisualizer,
    RandomGenerator,
    Simulator,
    StartMeshcat
)

from manipulation import running_as_notebook
from manipulation.scenarios import AddIiwaDifferentialIK
from manipulation.station import MakeHardwareStation

from constants import *
from setup import construct_scenario
from planner import Planner


class NoDiffIKWarnings(logging.Filter):
    def filter(self, record):
        return not record.getMessage().startswith("Differential IK")


logging.getLogger("drake").addFilter(NoDiffIKWarnings())

In [2]:
meshcat = StartMeshcat()
rng = np.random.default_rng(135)  # this is for python
generator = RandomGenerator(rng.integers(0, 1000))  # this is for c++

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
def jengabot_demo():
    meshcat.Delete()
    builder = DiagramBuilder()
    scenario = construct_scenario()
    station = builder.AddSystem(MakeHardwareStation(scenario, package_xmls=['./package.xml']))
    plant = station.GetSubsystemByName("plant")

    # Wire planner inputs
    planner = builder.AddSystem(Planner(plant))
    builder.Connect(station.GetOutputPort("wsg.state_measured"), planner.GetInputPort("wsg_state"))
    builder.Connect(station.GetOutputPort("iiwa.position_measured"), planner.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("body_poses"), planner.GetInputPort("body_poses"))

    # Wire diff IK
    robot = station.GetSubsystemByName("iiwa.controller").get_multibody_plant_for_control()
    diff_ik = AddIiwaDifferentialIK(builder, robot)
    builder.Connect(planner.GetOutputPort("X_WG"), diff_ik.get_input_port(0))
    builder.Connect(station.GetOutputPort("iiwa.state_estimated"), diff_ik.GetInputPort("robot_state"))
    builder.Connect(planner.GetOutputPort("use_robot_state"), diff_ik.GetInputPort("use_robot_state"))
    builder.Connect(planner.GetOutputPort("wsg_position"), station.GetInputPort("wsg.position"))
    builder.Connect(diff_ik.get_output_port(), station.GetInputPort("iiwa.position"))

    visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)
    diagram = builder.Build()

    simulator = Simulator(diagram)
    simulator.AdvanceTo(0.1)
    meshcat.Flush()  # Wait for the large object meshes to get to meshcat.

    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
        meshcat.DeleteButton("Stop Simulation")

In [4]:
jengabot_demo()

INFO:drake:Meshcat listening for connections at http://localhost:7001


[0. 1.] [[0 0]]
Press Escape to stop the simulation
[1.1 2.1] [[0 0]]
[2.2 3.2] [[0 0]]
[3.3 4.3] [[0 0]]
[4.4 5.4] [[0 0]]
[5.5 6.5] [[0 0]]
[6.6 7.6] [[0 0]]
[7.7 8.7] [[0 0]]


KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8e10b301-2776-448f-be43-c7f6fb54fa1f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>